In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set()
from matplotlib.backends.backend_pdf import PdfPages
#from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.colors as col
import matplotlib as mpl
import numpy as np #import for transposing
import math
import pylab as pl
import mpld3 as ml
import math
import import_ipynb
from sklearn.metrics import mean_squared_error,mean_absolute_error
from function_log import add_tdate,get_week,density_interp,extract_date,qtr_comp,IC_run_sim,new_compare,stoch_model,run_sim

from scipy.optimize import minimize

importing Jupyter notebook from function_log.ipynb


In [2]:
def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

In [3]:
new_census = pd.read_csv('new_census.csv')
new_census['Date'] = pd.to_datetime(new_census['Date'])
new_census['ObsYear'] = new_census.Date.apply(lambda x: x.year)
new_census['ObsMonth'] = new_census.Date.apply(lambda x:x.month)

new_census.T = pd.to_numeric(new_census['T'],errors='coerce')
new_census.J = pd.to_numeric(new_census.J)
new_census.A = pd.to_numeric(new_census.A)
new_census.S = pd.to_numeric(new_census.S)

In [4]:
t_types = pd.read_csv('troop_types.csv')

rural_troops = list(t_types.loc[t_types['Type Code'] == 0]['Troop'])
rural = new_census.loc[new_census.Troop.isin(rural_troops)]

urb_troops = list(t_types.loc[t_types['Type Code'] == 1]['Troop'])
urban = new_census.loc[new_census.Troop.isin(urb_troops)]

In [ ]:
from __future__ import print_function
import sys
import threading
from time import sleep
try:
    import thread
except ImportError:
    import _thread as thread

def quit_function():
        # print to stderr, unbuffered in Python 2.
        print('Parameter set rejected', file=sys.stderr)
        sys.stderr.flush() # Python 3 stderr is likely buffered.
        thread.interrupt_main() # raises KeyboardInterrupt
        

def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            finally:
                timer.cancel()
            return result
        return inner
    return outer

##Initialize timeout function
@exit_after(300)
def min_func(paramset):
    print(paramset)
    try:    
        ##Extract median NRMSE value from model run
        res_df = new_compare(rural,iters=25,
                             fus_fis=paramset[0],
                             no_ev=paramset[1],
                             rt_chg=paramset[2],
                             b_j=paramset[3],
                             d_j=paramset[4],
                             d_a=paramset[5],
                             ae_j=paramset[6],
                             ae_s=paramset[7],
                             ae_a=paramset[8],
                             t_p=paramset[9],
                             por=paramset[10],
                             fsr=paramset[11],
                             thresh=paramset[12],
                             fis_thresh=paramset[13],
                             lit_size=paramset[14],
                             dfe='off'
                            )
        smape = res_df.loc[res_df.sMAPE<=100]
        smape = smape.loc[:,'sMAPE'].dropna()

        total = len(smape)
        good = len(smape.loc[smape <= 15])
        perc = 1-(good/total)

        return perc
    
    except:
        return 5

In [ ]:
##USE PYSWARM HERE
from pyswarm import pso

#x0 = [0.4,0.8,0.1,0.299,0.345,0.216,0.143,0.5,0.5,0.5,0.3,0.2,0.4,4,45,4]

lb = [0.1,.8,.05,.254,.278,.136,.034,.1,.1,.1,.15,.1,.2,4,38,3]
ub = [0.7,.95,.5,.344,.412,.296,.252,.8,.8,.8,.70,.6,.8,6,68,5]

xopt,fopt = pso(min_func,lb,ub,maxiter=2,debug=True,minfunc=0.01)

No constraints given.
[  0.52646334   0.93190353   0.08254146   0.33672639   0.36750124
   0.25102827   0.21525454   0.56566029   0.65675022   0.70110681
   0.16349154   0.48312809   0.21801659   5.20618596  63.48687469
   3.42019025]


function_log.ipynb:63: RuntimeWarning: invalid value encountered in double_scalars
  "outputs": [],
function_log.ipynb:63: RuntimeWarning: divide by zero encountered in double_scalars
  "outputs": [],


[  0.58360796   0.82740769   0.14910157   0.33988733   0.33412962
   0.20873931   0.07618506   0.70815268   0.58851648   0.42730269
   0.2275241    0.4997896    0.37203743   4.51773474  67.12223234
   4.04220283]
[  0.20184827   0.88489302   0.29848843   0.32191444   0.34515339
   0.27670305   0.24652185   0.46592201   0.5095421    0.7441209
   0.31372514   0.40878107   0.33906803   5.35989731  41.89810014
   3.00731098]


Parameter set rejected


In [ ]:
xopt